In [7]:
import h5py
import scipy.io as io
import PIL as Image
# import PIL.Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
from scipy import spatial
import scipy
import json
from matplotlib import cm as CM
#from Image import *
from tqdm import tqdm
import pandas as pd
import csv 

## Step 1: Build Dataset for Training Classifier 

#### Get QNRF Dataset crowd count to use them for splitting into different categories (Sparse and Dense) 

In [120]:
output = pd.DataFrame(columns=['URL', 'GroundTruth'])

In [121]:
paths = ['/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train',
          '/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Test']

img_paths = []
for path in paths:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
    print("Files:"+str(len(img_paths)))

for img in tqdm(img_paths):
    # print(img)
    mat = io.loadmat(img.replace('.jpg','_ann.mat'))
    ground_truth = len(mat["annPoints"])
    # print(ground_truth)
    output = output.append({'URL':img, 'GroundTruth':ground_truth},ignore_index=True)
output_path = os.path.abspath(os.path.join(paths[0],'..'))+"/qnrf_counts.csv"

  2%|▏         | 30/1535 [00:00<00:05, 293.73it/s]

Files:1201
Files:1535


100%|██████████| 1535/1535 [00:05<00:00, 286.39it/s]


#### Store labelled counts in CSV file (ground truth)

In [122]:
output.to_csv(output_path,index=False,quoting=csv.QUOTE_ALL)

#### Get Shaghai Tech counts to use them for splitting into different categories (Sparse and Dense) 

In [123]:
root = '/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/'
part_A_test = os.path.join(root,'part_A_final/test_data','images')
part_A_train = os.path.join(root,'part_A_final/train_data','images')
part_B_test = os.path.join(root,'part_B_final/test_data','images')
part_B_train = os.path.join(root,'part_B_final/train_data','images')
path_sets = [part_A_train,part_A_test,part_B_train,part_B_test]

In [124]:
# List of all image paths
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
    print(len(img_paths))

300
482
882
1198


In [125]:
output2 = pd.DataFrame(columns=['URL', 'GroundTruth'])
for img in tqdm(img_paths):
    mat = io.loadmat(img.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    # print(img)
    ground_truth = int(mat['image_info'][0,0][0,0][1])
    # print(ground_truth)
    output2 = output2.append({'URL':img, 'GroundTruth':ground_truth},ignore_index=True)
output_path2 = os.path.abspath(root)+"/shaghaitech_counts.csv"

100%|██████████| 1198/1198 [00:03<00:00, 304.24it/s]


In [126]:
output2.to_csv(output_path2,index=False,quoting=csv.QUOTE_ALL)

##### Concat and make data folders for each class

In [127]:
final_output = output.append(output2, ignore_index=True)
print(len(final_output))
print(type(final_output))

2733
<class 'pandas.core.frame.DataFrame'>


In [129]:
print(type(sparse_urls['URL']))

<class 'pandas.core.series.Series'>


### Create Dense and Sparse splits using a common threshold 

Note: This threshold is chosen arbitrarily, slightly less than avg. count of shanghai part B images set (which have mostly sparse images). 

In [141]:
print(len(final_output[final_output["GroundTruth"]<=100]))
print(len(final_output[final_output["GroundTruth"]>100]))
sparse_urls = final_output[final_output["GroundTruth"]<=100]
dense_urls = final_output[final_output["GroundTruth"]>100]

dense_path = '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/'
sparse_path = '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/'

for i,x in enumerate(sparse_urls['URL']):
    dest= x.replace('.jpg', '_sparse'+str(i)+'.jpg')
    !cp {x} {dest}
    !mkdir -p {sparse_path} && mv -v {dest} {sparse_path}
for i,x in enumerate(dense_urls['URL']):
    dest= x.replace('.jpg', '_dense'+str(i)+'.jpg')
    !cp {x} {dest}
    !mkdir -p {dense_path} && mv -v {dest} {dense_path}

!ls {dense_path} | wc -l
!ls {sparse_path} | wc -l

420
2313
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train/img_0171_sparse0.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/img_0171_sparse0.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train/img_0090_sparse1.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/img_0090_sparse1.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train/img_1007_sparse2.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/img_1007_sparse2.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train/img_0754_sparse3.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/img_0754_sparse3.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/flickr_latest/UCF-QNRF_ECCV18/Train/img_0205_sparse4.jpg' -> '/home/ubuntu/mayub

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/train_data/images/IMG_232_sparse38.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_232_sparse38.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/train_data/images/IMG_239_sparse39.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_239_sparse39.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/train_data/images/IMG_256_sparse40.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_256_sparse40.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/train_data/images/IMG_61_sparse41.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_61_sparse41.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_A_final/train_data/images/IMG_125_sparse4

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_84_sparse76.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_84_sparse76.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_323_sparse77.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_323_sparse77.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_288_sparse78.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_288_sparse78.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_209_sparse79.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_209_sparse79.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_345_sparse8

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_79_sparse113.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_79_sparse113.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_48_sparse114.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_48_sparse114.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_351_sparse115.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_351_sparse115.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_52_sparse116.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_52_sparse116.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_301_spa

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_78_sparse150.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_78_sparse150.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_363_sparse151.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_363_sparse151.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_178_sparse152.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_178_sparse152.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_294_sparse153.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_294_sparse153.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_161

...

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_149_dense2132.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_149_dense2132.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_336_dense2133.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_336_dense2133.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_44_dense2134.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_44_dense2134.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_247_dense2135.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_247_dense2135.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/train_data/images/IMG_190_den

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_35_dense2169.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_35_dense2169.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_316_dense2170.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_316_dense2170.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_40_dense2171.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_40_dense2171.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_173_dense2172.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_173_dense2172.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_283_dense2173.

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_207_dense2207.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_207_dense2207.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_110_dense2208.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_110_dense2208.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_74_dense2209.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_74_dense2209.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_291_dense2210.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_291_dense2210.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_244_dense221

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_109_dense2244.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_109_dense2244.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_232_dense2245.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_232_dense2245.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_176_dense2246.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_176_dense2246.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_51_dense2247.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_51_dense2247.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_242_dense224

'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_132_dense2282.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_132_dense2282.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_243_dense2283.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_243_dense2283.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_127_dense2284.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_127_dense2284.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_306_dense2285.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_306_dense2285.jpg'
'/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/shanghaitech/part_B_final/test_data/images/IMG_23_dense22

### Create Train,Test, Val Splits from temp folders

In [161]:
den = !ls -d -1 {dense_path+'*'}
spr = !ls {sparse_path+'*'}
den_df = pd.DataFrame(den,columns=["Url"])
spr_df = pd.DataFrame(spr,columns=["Url"])
print(len(den_df))
print(den_df)

2313
                                                    Url
0     /home/ubuntu/mayub/Github/transfer-learning-ke...
1     /home/ubuntu/mayub/Github/transfer-learning-ke...
2     /home/ubuntu/mayub/Github/transfer-learning-ke...
3     /home/ubuntu/mayub/Github/transfer-learning-ke...
4     /home/ubuntu/mayub/Github/transfer-learning-ke...
5     /home/ubuntu/mayub/Github/transfer-learning-ke...
6     /home/ubuntu/mayub/Github/transfer-learning-ke...
7     /home/ubuntu/mayub/Github/transfer-learning-ke...
8     /home/ubuntu/mayub/Github/transfer-learning-ke...
9     /home/ubuntu/mayub/Github/transfer-learning-ke...
10    /home/ubuntu/mayub/Github/transfer-learning-ke...
11    /home/ubuntu/mayub/Github/transfer-learning-ke...
12    /home/ubuntu/mayub/Github/transfer-learning-ke...
13    /home/ubuntu/mayub/Github/transfer-learning-ke...
14    /home/ubuntu/mayub/Github/transfer-learning-ke...
15    /home/ubuntu/mayub/Github/transfer-learning-ke...
16    /home/ubuntu/mayub/Github/transfer-le

In [162]:
# 90-10-10 Split
train, validate, test = np.split(den_df.sample(frac=1), [int(.8*len(den_df)), int(.9*len(den_df))])
train2, validate2, test2 = np.split(spr_df.sample(frac=1), [int(.8*len(spr_df)), int(.9*len(spr_df))])

In [163]:
# Combine splits
print(len(train))
print(len(test))
print(len(validate))
print(len(train2))
print(len(test2))
print(len(validate2))
final_train = train.append(train2,ignore_index=True)
final_test = test.append(test2,ignore_index=True)
final_validate = validate.append(validate2,ignore_index=True)

1850
232
231
336
42
42


#### Copy the actual images to temp folders

In [165]:
train_path = '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/'
test_path = '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/test/'
validate_path = '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/'

#### Copy also the duplicate filenames as numbered
for x in final_train['Url']:
    !mkdir -p {train_path} && cp -v -r {x} {train_path}
for x in final_test['Url']:
    !mkdir -p {test_path} && cp -v -r {x} {test_path}
for x in final_validate['Url']:
    !mkdir -p {validate_path} && cp -v -r {x} {validate_path}

!ls {train_path} | wc -l
!ls {test_path} | wc -l
!ls {validate_path} | wc -l

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0954_dense541.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0954_dense541.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0679_dense48.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0679_dense48.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0118_dense769.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0118_dense769.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0333_dense883.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0333_dense883.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_121_dense1814.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_121_dense1814.jpg'
'/home/

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_12_dense1711.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_12_dense1711.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_379_dense2155.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_379_dense2155.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_40_dense2171.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_40_dense2171.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0989_dense398.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0989_dense398.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_243_dense2283.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_243_dense2283.jpg'
'/home/ub

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_269_dense1756.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_269_dense1756.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_93_dense2242.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_93_dense2242.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_188_dense1702.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_188_dense1702.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0927_dense1024.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0927_dense1024.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0748_dense622.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0748_dense622.jpg'
'/hom

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_137_dense2005.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_137_dense2005.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0533_dense163.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0533_dense163.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0186_dense331.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0186_dense331.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_246_dense1761.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_246_dense1761.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_1160_dense174.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_1160_dense174.jpg'
'/hom

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0347_dense726.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0347_dense726.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_307_dense2176.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_307_dense2176.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_93_dense1662.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/IMG_93_dense1662.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0055_dense268.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0055_dense268.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0858_dense703.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/train/img_0858_dense703.jpg'
'/home/

...

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0827_dense312.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0827_dense312.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0932_dense3.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0932_dense3.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0776_dense791.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0776_dense791.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_1058_dense654.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_1058_dense654.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0147_dense1318.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0147_dense13

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_31_dense2045.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_31_dense2045.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0236_dense1382.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0236_dense1382.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0842_dense247.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0842_dense247.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_1069_dense957.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_1069_dense957.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0952_dense829.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0952_dens

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0167_dense1202.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0167_dense1202.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_330_dense2056.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_330_dense2056.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/IMG_42_dense1965.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_42_dense1965.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_1035_dense183.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_1035_dense183.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0022_dense1392.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0022_den

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0073_dense1443.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0073_dense1443.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_1185_dense284.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_1185_dense284.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0221_dense1485.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0221_dense1485.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0359_dense886.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0359_dense886.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/dense/img_0068_dense1421.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/img_0068

'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_35_sparse52.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_35_sparse52.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_390_sparse224.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_390_sparse224.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_125_sparse414.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_125_sparse414.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_85_sparse144.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_85_sparse144.jpg'
'/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/sparse/IMG_267_sparse383.jpg' -> '/home/ubuntu/mayub/Github/transfer-learning-keras/dataset/crowddata/validate/IMG_267_sparse

#### Step 1b) Run the  `build_dataset.py` script with the above folders to get clas label folders.

#### Step 2) Extract features for each Image 

#### Run  `extract_features.py` script to get features for each class image (train, test, val) which generates csv files.

#### Step 3) Train Classifier on extracted Image features

#### Run `train.py` to train a simple multi-class logistic regression model on generated features (csv files).

### Saving a VGG top-less model for serving

In [1]:
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
model = VGG16(weights="imagenet", include_top=False)

In [17]:
model.save('vgg16_nh')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: vgg16_nh/assets


## Test after serving the model

In [2]:
import grpc
# from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
# from tensorflow_serving.apis import prediction_service_pb2
from tensorflow.core.framework import types_pb2
import tensorflow as tf
import PIL as Image
import joblib

In [4]:
def prepare_image(img, im_type=None):
    if im_type=="classify":
        newsize = (224, 224) 
        img = img.resize(newsize) 
    #Function to load,normalize and return image 
    im = np.array(img)
    im = im/255.0
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225
    im = np.expand_dims(im,axis  = 0)
    print(str(im.shape))
    return im

In [22]:
#image = Image.Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/sparse/IMG_7.jpg')
image = Image.Image.open('/home/ubuntu/CrowdSourcing_Projects/CSRNet-keras/data/Classify_all/dense/IMG_15.jpg')

image = prepare_image(image,im_type='classify')

(1, 224, 224, 3)


In [42]:
image.shape

(1, 224, 224, 3)

##### Start the Tensorflow Model server before runnig this

In [43]:
channel = grpc.insecure_channel("localhost:8500")
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
# stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
request = predict_pb2.PredictRequest()
request.model_spec.name = "vgg16_nohead"
request.inputs["input_1"].CopyFrom(tf.make_tensor_proto(image, dtype=types_pb2.DT_FLOAT))

In [45]:
type(request.inputs)

google.protobuf.pyext._message.MessageMapContainer

In [41]:
response = stub.Predict(request, timeout=60.0)
response

outputs {
  key: "block5_pool"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 7
      }
      dim {
        size: 7
      }
      dim {
        size: 512
      }
    }
    float_val: 1.2084078788757324
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.15034158527851105
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.5272638201713562
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.6518181562423706
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.012457258999347687
    float_val: 0.0
    float_val: 0.13896651566028595
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 0.0
    float_val: 2.131284475326538
    float_val: 0.0
    float_val: 0.27982309460639954
    float_val: 0.0
    float_val: 0.7641890048980713
    float_val: 0.0
    float_val: 0.5584

In [25]:
print(type(response.outputs['block5_pool']))
print(tf.make_ndarray(response.outputs['block5_pool']).shape)

<class 'tensorflow.core.framework.tensor_pb2.TensorProto'>
(1, 7, 7, 512)


#### Load the trained classifier model

In [3]:
clf = joblib.load('/home/ubuntu/mayub/Github/transfer-learning-keras/output/classify_model.pkl')

In [26]:
### Get features from VGG16 (no head) model
feat = tf.make_ndarray(response.outputs['block5_pool'])
features = feat.reshape((feat.shape[0], 7 * 7 * 512))
features.shape

(1, 25088)

In [4]:
### Make prediction using extracted features
class_label = clf.predict(features)

NameError: name 'features' is not defined

In [32]:
print(class_label)

['0']


### Class 0 - Dense 
### Class 1 - Sparse